## Pandas analysis

In the following a series of exercises is proposed on a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGA's. Each measurement (i.e. each raw) consists of the address of the TDC providing the signal, 'FPGA' and 'TDC_Channel, and the timing information itself, 'ORBIT_CNT', 'BX_COUNTER' and 'TDC_MEAS'. Each TDC count correspond 25/30 ns, whereas the BX_COUNTER feauters gets updated every 25 ns and the ORBIT_CNT every 'x' BX_COUNTER. You can see these way of storing the time as similar to hours, minutes and seconds.


1. Create a Pandas DataFrame by read N raws of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of raws and larger that 10k.

In [50]:
import pandas as pd
import numpy as np

file_name = "raw_data/data_000637.txt"
df = pd.read_csv(file_name, nrows=10000)
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
5,1,0,63,3869200167,2762,4
6,1,0,61,3869200167,2772,14
7,1,0,139,3869200167,2776,0
8,1,0,62,3869200167,2774,21
9,1,0,60,3869200167,2788,7


2\. Find out the value of 'x'

In [115]:
Xs = []
bx_start = df.loc[0,'BX_COUNTER']
for i in range(1,len(df)):
    if df.loc[i,'ORBIT_CNT'] != df.loc[i-1,'ORBIT_CNT']:
        Xs.append(df.loc[i-1,'BX_COUNTER'])
        bx_start = df.loc[i,'BX_COUNTER']
print(Xs)

# BOH

[3187, 3538, 2766, 3377, 3538, 3521, 3247, 2985, 3542, 3484, 3242, 1365, 1935, 3542, 3375, 3511, 2310, 3480, 3337, 3475, 3009, 2214, 3390, 3529, 3481, 3562, 3493, 2576, 3557, 6, 3563, 3516, 2094, 2619, 3532, 3426, 3550, 2966, 3519, 3434, 3526, 1739, 3563, 3384, 3489, 3249, 3552, 4, 3552, 5, 3556, 3527, 3510, 3557, 3, 3563, 3403, 3555, 3076, 3477, 3479, 2698, 3181, 3387, 3547, 3350, 3299, 3554, 3189, 3542, 2056, 2750, 3461, 3537, 3510, 3446, 3530, 3492, 3362, 3357, 3136, 3492, 3238, 3141, 1739, 3541, 3038, 3550, 0, 3563, 1073, 2279, 3529, 3408, 3546, 3458, 3327, 3505, 3538, 3256, 3027, 3447, 3056, 3376, 3332, 3183, 3422, 3065, 3480, 1851]


3\. Find out how much the data taking lasted. You can either make an estimate on the baseis of the fraction of the measurements (raws) you read, or perform this check precisely by reading out the whole dataset

4\. Create a new column with the actual time in ns (as a combination of the other three columns with timing information)

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1

6\. Create a new DataFrame with only the raws with HEAD=1

7\. Make two occupancy plots (one per FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139